In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
import pyart
import numpy as np
from glob import glob
import datetime as dt
import os
import sys
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cf
from pyproj import Proj as Proj

from pyMRMS import retrieveMRMS,readFL

In [ ]:
getMRMS = True
strtMRMS='20170430-180000'
endMRMS='20170430-200000'
mrmsSaveDir='/Users/danstechman/Desktop/MRMS/'

pltFT = True
flFile = '/Users/danstechman/GoogleDrive/VSE-Data/flightLevel/20170430H1_AC.nc'
strtFL = strtMRMS
endFL = endMRMS


In [ ]:
if getMRMS:
    retrieveMRMS.fetch(strtMRMS,endMRMS,mrmsSaveDir)

In [ ]:
if pltFT:
    flData = readFL.getP3(flFile,strtDT=strtFL,endDT=endFL)